**Họ tên: Vương Trí Thiên Công**

**Mã sinh viên: 18020240**

**Lớp: K63-K2**

In [ ]:
! pip install py7zr
import py7zr
with py7zr.SevenZipFile('/kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z', mode='r') as z:
    z.extractall()
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip
import numpy as np
import pandas as pd
import sklearn
from sklearn.utils import shuffle
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

>  # Workflow
1. <a href="#Problem-description">Problem description</a> 
2. <a href="#Data-preprocessing">Data preprocessing</a>
    * <a href="#Compare-TRAIN-and-TEST-dataset">Compare TRAIN and TEST dataset</a>
    * <a href="#Price-distribution">Price distribution</a>
    * <a href="#Data-cleansing">Data cleansing</a>
    * <a href="#Preprocessor-function">Preprocessor function</a>
    * <a href="#Text-encoding">Text encoding</a>
    * <a href="#Evaluate">Evaluate</a>
3. <a href="#Model-training">Model training</a>
    * <a href="#Linear-Regression">Linear Regression</a>     
    * <a href="#Ridge-Regression">Ridge Regression</a>     
    * <a href="#SGD-Regressor">SGD Regressor</a>
    * <a href="#Ensemble-models-(Boosting-method)">Ensemble models (Boosting method)</a>
4. <a href="#Conclusion">Conclusion</a>
5. <a href="#Result">Result</a>

# Problem description

Dự đoán giá của sản phẩm.

**Input:** Dữ liệu dạng text về **tên**, **tình trạng**, **danh mục**, **nhãn hiệu**, **shipping** và **nhận xét của khách hàng** về sản phẩm.

**Output:** Giá của sản phẩm (>0) và được Kaggle đánh giá theo công thức:

<a href="https://www.codecogs.com/eqnedit.php?latex=RMSLE&space;=&space;\sqrt{&space;\frac{1}{N}\sum_{i=1}^{N}&space;(log(\hat{y}_{i}&plus;1)-&space;log(y_{i}&plus;1))^2}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?RMSLE&space;=&space;\sqrt{&space;\frac{1}{N}\sum_{i=1}^{N}&space;(log(\hat{y}_{i}&plus;1)-&space;log(y_{i}&plus;1))^2}" title="RMSLE = \sqrt{ \frac{1}{N}\sum_{i=1}^{N} (log(\hat{y}_{i}+1)- log(y_{i}+1))^2}" /></a>

trong đó:

* $\hat{y}_{i}$: giá dự đoán

* $y_{i}$: giá thực sự

**Kết luận:** dạng bài toán hồi quy.

# Data preprocessing
Tập train data với hơn 1 triệu điểm dữ liệu gồm 7 cột features và 1 cột giá (**price**).

Trộn tập dữ liệu với random_state=0.

In [ ]:
train_data = pd.read_table('../working/train.tsv')
train_data = shuffle(train_data, random_state=0)
print(train_data.shape)
# train_data = train_data[:100]
# print(train_data.shape)
train_data.head()

Tập test data thì không có cột **price** vì đây là giá trị cần dự đoán để submit.

In [ ]:
test_data = pd.read_table('../working/test_stg2.tsv')
print(test_data.shape)
# test_data = test_data[:100]
test_data.head()

## Compare TRAIN and TEST dataset

**Vấn đề**: Nếu 2 tập dữ liệu train và test quá khác nhau, việc học trên tập train sẽ không mang lại ý nghĩa khi cần dự đoán trên tập test.

Vậy nên trước khi phân tích sâu hơn, ta cần kiểm tra độ tương đồng của tập train và test.

**Giải pháp**: So sánh **phần trăm** các giá trị chiếm đa số trong một vài feature để có cái nhìn cơ bản về tập dữ liệu ta cần làm việc cùng.

In [ ]:
print("Train dataset:\n" + str(train_data['item_condition_id'].value_counts() / train_data.shape[0]))
print("\nTest dataset:\n" + str(test_data['item_condition_id'].value_counts() / test_data.shape[0]))

5 giá trị **item_condition_id** trên tập train và test có tỷ lệ tương đồng.

In [ ]:
print("Train dataset:\n" + str(train_data['brand_name'].value_counts().head() / train_data.shape[0]))
print("\nTest dataset:\n" + str(test_data['brand_name'].value_counts().head() / test_data.shape[0]))

Giá trị **brand_name** trên tập train và test có tỷ lệ tương đồng.

In [ ]:
print("Train dataset:\n" + str(train_data['shipping'].value_counts() / train_data.shape[0]))
print("\nTest dataset:\n" + str(test_data['shipping'].value_counts() / test_data.shape[0]))

2 giá trị **shipping** trên tập train và test có tỷ lệ tương đồng.

### Kết luận về tập dữ liệu

Tập train và test có dữ liệu cân bằng, thuận lợi cho việc train model và dự đoán.

**Để chuẩn bị dữ liệu một cách tốt nhất, ta cần khái quát các features và đề xuất cách xử lý:**
* **name** và **item_description**: dùng các kỹ thuật xử lý ngôn ngữ tự nhiên
* **item_condition_id**, **shipping** và **brand_name**: các features này chỉ xuất hiện một vài giá trị nên sẽ chia thành các label
* **category_name**: feature này có dạng A/B/C nên dự định sẽ tách riêng ra làm 3 phần rồi xử lý theo dạng label

## Price distribution

Khảo sát giá sản phẩm cho thấy sự không cân xứng (giá đồ tiêu dùng so với trang sức).

In [ ]:
import seaborn

seaborn.distplot(train_data['price'])

Hàm đồng biến log() sẽ đưa đồ thị giá về gần dạng phân bố chuẩn.

In [ ]:
seaborn.distplot(np.log1p(train_data.price))

**Vấn đề:** Kết quả đánh giá của kaggle dựa trên logarit của giá sản phẩm nên bắt buộc giá trả về phải lớn hơn 0.

Qua một số lần thử nghiệm thì số lần dự đoán giá bị âm khá nhiều, dẫn đến khi lấy log() bị lỗi.

**Giải pháp:** dự đoán log() của giá rồi qua hàm exp() sẽ trả về giá trị luôn dương.


In [ ]:
train_data['log_price'] = np.log1p(train_data.price)
train_data.iloc[0]

Tập dữ liệu train giờ đã có thêm cột log_price.

## Data cleansing

Cột danh mục sản phẩm (**category_name**) đa số ở dạng top/sub/item.

Ta thêm 3 cột vào dữ liệu: **top**, **sub** và **item** để dễ dàng phân tích text.

In [ ]:
def split_cat(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Others', 'Others', 'Others']
train_data['cat_top'], train_data['cat_sub'], train_data['cat_item'] = zip(*train_data['category_name'].apply(lambda x: split_cat(x)))
test_data['cat_top'], test_data['cat_sub'], test_data['cat_item'] = zip(*test_data['category_name'].apply(lambda x: split_cat(x)))

### NaN values 

**Vấn đề:** Giá trị NaN sẽ gây ra lỗi khi chuyển hóa dữ liệu dạng text về vector.

**Giải pháp:** Tìm trên 2 tập dữ liệu những features nào chứa giá trị NaN rồi thay thế chúng.

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

Thay thế NaN thành giá trị null trên 3 features: **category**, **brand** và **item_description**

In [ ]:
train_data['category_name'] = train_data['category_name'].fillna(value='Null')
train_data['brand_name'] = train_data['brand_name'].fillna(value='Null')
train_data['item_description'] = train_data['item_description'].fillna(value='Null')

test_data['category_name'] = test_data['category_name'].fillna(value='Null')
test_data['brand_name'] = test_data['brand_name'].fillna(value='Null')
test_data['item_description'] = test_data['item_description'].fillna(value='Null')

## Preprocessor function 

Hàm làm sạch text giúp cải thiện két quả khá nhiều (giảm 0.1 loss ở kết quả submib Kaggle ~ cao hơn 100 vị trí so với không dùng)

Nhiệm vụ:
* Xóa thẻ tag và các ký tự đặc biệt.
* Chuyển ký tự viết hoa thành thường.

In [ ]:
import re
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["] using the resub method:
    text = re.sub(r'\\', '', text)

    text = re.sub(r'\"', '', text)   

    text = re.sub(r'\'', '', text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

# Example
clean_text("<html>This is is not a\" sentence.<\html>").split()

## Text encoding

Dựa vào số lượng mẫu của các đặc tính (unique number of each feature) ta chia dữ liệu thành 2 dạng để xử lý:
* Biến đổi về dạng vector số: **name**, **item_description**.
* Phân loại theo các label: **item_condition_id**, **brand_name**, **shipping**, **cat_top**, **cat_sub**, **cat_item**.

In [ ]:
train_data.nunique()

### Vectorizer

Mô hình **Bag of words**: biểu diễn mỗi mẫu dữ liệu dưới dạng một vector số trong đó mỗi chiều là một từ cụ thể trong kho dữ liệu.

Kho dữ liệu được khởi tạo khi gọi lệnh fit_transform từ tập train.

Đối với tập test thì chỉ việc biểu diễn vector theo kho dữ liệu đã có (transform).

Những từ như and, a, the, ... (stopwords) là những từ không có ý nghĩa và xuất hiện nhiều khiến lu mờ các từ khác sẽ được loại bỏ.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english",
                            preprocessor=clean_text)

# train_name = vectorizer.fit_transform(train_data['name'])
# test_name = vectorizer.transform(test_data['name'])
# pickle.dump(train_name, open("train_name.pickle", "wb"))
# pickle.dump(test_name, open("test_name.pickle", "wb"))

train_name = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_name.pickle'), 'rb'))
test_name = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_name.pickle'), 'rb'))

**Vấn đề:** Đối với phần nhận xét của khách hàng **item_description** thì cần cách xử lý khác Bag of words bởi:
* Những từ xuất hiện với tần suất nhiều sẽ làm lu mờ ý nghĩa của những từ quan trọng.

**Giải pháp:** Cách xử lý là dùng mô hình **TF-IDF**: vector biểu diễn điểm dữ liệu sẽ dựa vào độ quan trọng của từ.

Độ quan trọng được tính theo công thức:

<a href="https://www.codecogs.com/eqnedit.php?latex=i&space;*log(\frac{n}{1&space;&plus;&space;n_{i}})" target="_blank"><img src="https://latex.codecogs.com/gif.latex?i&space;*log(\frac{n}{1&space;&plus;&space;n_{i}})" title="i *log(\frac{n}{1 + n_{i}})" /></a>

trong đó:
* $i$: là số lần từ xuất hiện trong câu
* $n$: là tổng số điểm dữ liệu
* $n_{i}$: là số điểm dữ liệu có từ đó

Thay vì tạo kho dữ liệu với từ riêng lẻ (unigrams: 'do', 'not') thì sẽ đi kèm với 2 từ (bigrams: 'do', 'not', 'do-not').

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             ngram_range=(1, 2))

# train_des = tfidf.fit_transform(train_data['item_description'])
# test_des = tfidf.transform(test_data['item_description'])
# pickle.dump(train_des, open("train_des.pickle", "wb"))
# pickle.dump(test_des, open("test_des.pickle", "wb"))

train_des = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_des.pickle'), 'rb'))
test_des = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_des.pickle'), 'rb'))

### Các features phân loại theo label

Số lượng giá trị unique của các feature sau khá ít khi so với số điểm dữ liệu (1,482,535) nên ta vector hóa nó theo các label.

Number of unique elements in **brand**: 4810

Có thể coi đây không phải một số lớn khi so với 1 triệu 4 điểm dữ liệu.

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb_brand = LabelBinarizer(sparse_output=True)

# train_brand = lb_brand.fit_transform(train_data['brand_name'])
# test_brand = lb_brand.transform(test_data['brand_name'])
# pickle.dump(train_brand, open("train_brand.pickle", "wb"))
# pickle.dump(test_brand, open("test_brand.pickle", "wb"))

train_brand = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_brand.pickle'), 'rb'))
test_brand = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_brand.pickle'), 'rb'))

Number of unique elements in **condition_id**: 5

In [ ]:
lb_condition_id = LabelBinarizer(sparse_output=True)

# train_condition_id = lb_condition_id.fit_transform(train_data['item_condition_id'])
# test_condition_id = lb_condition_id.transform(test_data['item_condition_id'])
# pickle.dump(train_condition_id, open("train_condition_id.pickle", "wb"))
# pickle.dump(test_condition_id, open("test_condition_id.pickle", "wb"))

train_condition_id = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_condition_id.pickle'), 'rb'))
test_condition_id = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_condition_id.pickle'), 'rb'))

Number of unique elements in **shipping**: 2

In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)

# train_shipping = lb_shipping.fit_transform(train_data['shipping'])
# test_shipping = lb_shipping.transform(test_data['shipping'])
# pickle.dump(train_shipping, open("train_shipping.pickle", "wb"))
# pickle.dump(test_shipping, open("test_shipping.pickle", "wb"))

train_shipping = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_shipping.pickle'), 'rb'))
test_shipping = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_shipping.pickle'), 'rb'))

Number of unique elements in **cat_top**: 11

In [ ]:
lb_cat_top = LabelBinarizer(sparse_output=True)

# train_cat_top = lb_cat_top.fit_transform(train_data['cat_top'])
# test_cat_top = lb_cat_top.transform(test_data['cat_top'])
# pickle.dump(train_cat_top, open("train_cat_top.pickle", "wb"))
# pickle.dump(test_cat_top, open("test_cat_top.pickle", "wb"))

train_cat_top = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_cat_top.pickle'), 'rb'))
test_cat_top = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_cat_top.pickle'), 'rb'))

Number of unique elements in **cat_sub**: 113

In [ ]:
lb_cat_sub = LabelBinarizer(sparse_output=True)

# train_cat_sub = lb_cat_sub.fit_transform(train_data['cat_sub'])
# test_cat_sub = lb_cat_sub.transform(test_data['cat_sub'])
# pickle.dump(train_cat_sub, open("train_cat_sub.pickle", "wb"))
# pickle.dump(test_cat_sub, open("test_cat_sub.pickle", "wb"))

train_cat_sub = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_cat_sub.pickle'), 'rb'))
test_cat_sub = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_cat_sub.pickle'), 'rb'))

Number of unique elements in **cat_item**: 870

In [ ]:
lb_cat_item = LabelBinarizer(sparse_output=True)

# train_cat_item = lb_cat_item.fit_transform(train_data['cat_item'])
# test_cat_item = lb_cat_item.transform(test_data['cat_item'])
# pickle.dump(train_cat_item, open("train_cat_item.pickle", "wb"))
# pickle.dump(test_cat_item, open("test_cat_item.pickle", "wb"))

train_cat_item = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'train_cat_item.pickle'), 'rb'))
test_cat_item = pickle.load(open(os.path.join('/kaggle/input/trained-sparse-matrix', 'test_cat_item.pickle'), 'rb'))

## Evaluate

Vì bài toán giờ đã trở thành dự đoán log() và hàm đánh giá của kaggle là Root Mean Squared Log Error:

<a href="https://www.codecogs.com/eqnedit.php?latex=RMSLE&space;=&space;\sqrt{&space;\frac{1}{N}\sum_{i=1}^{N}&space;(log(\hat{y}_{i}&plus;1)-&space;log(y_{i}&plus;1))^2}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?RMSLE&space;=&space;\sqrt{&space;\frac{1}{N}\sum_{i=1}^{N}&space;(log(\hat{y}_{i}&plus;1)-&space;log(y_{i}&plus;1))^2}" title="RMSLE = \sqrt{ \frac{1}{N}\sum_{i=1}^{N} (log(\hat{y}_{i}+1)- log(y_{i}+1))^2}" /></a>

trong đó:

* $\hat{y}_{i}$: giá dự đoán

* $y_{i}$: giá thực sự

nên hàm evaluate có dạng như sau:

In [ ]:
def evaluate(preds, y_test):
    return np.sqrt(np.mean(np.power(preds - y_test, 2)))

Gói các features phục vụ quá trình training và testing.

Để đánh giá độ hiệu quả của model, ta lấy 20% tập train làm tập valid.

In [ ]:
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

train_features = (train_name, train_des, train_brand, train_condition_id, train_shipping, train_cat_top, train_cat_sub, train_cat_item)
X = hstack(train_features).tocsr()
X_train, X_test, y_train, y_test = train_test_split(X, train_data['log_price'], test_size=0.2, random_state=0)

test_features = (test_name, test_des, test_brand, test_condition_id, test_shipping, test_cat_top,  test_cat_sub, test_cat_item)
test_features = hstack(test_features).tocsr()

# Model training

3 models cổ điển được [sklearn](https://scikit-learn.org/stable/modules/classes.html#classical-linear-regressors) hỗ trợ.

In [ ]:
from sklearn.linear_model import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

rmsle_result = 100
best_model = ''

In [ ]:
def train(model, file_name):
    print('Start training ' + str(model).split('(')[0] + '...')
    model.fit(X_train, y_train)
    pickle.dump(model, open(file_name, 'wb'))
def loss(model, X_test, y_test):
    print('Root Mean Squared Error: ', end = '')
    result = evaluate(model.predict(X_test), y_test)
    print(result)
    return result
def load(file_name):
    model = pickle.load(open(os.path.join('/kaggle/input/trained-model', file_name), 'rb'))
    return model

## Linear Regression

Đầu tiên ta sẽ chọn loại model phổ biến và đơn giản nhất khi tiếp cận với bài toán regression: Hồi quy tuyến tính (trường hợp này là hồi quy đa bội: Multi linear regression).

![Multi linear regression](https://i.stack.imgur.com/nqI6I.png)

Áp dụng cho nghiên cứu mối quan hệ của nhiều biến độc lập và một biến phụ thuộc.

Phương trình tổng quát:

<a href="https://www.codecogs.com/eqnedit.php?latex=y&space;=&space;b_{0}&space;&plus;&space;b_{1}&space;*&space;x_{1}&space;&plus;&space;b_{2}&space;*&space;x_{2}&space;&plus;&space;...&space;&plus;&space;s" target="_blank"><img src="https://latex.codecogs.com/gif.latex?y&space;=&space;b_{0}&space;&plus;&space;b_{1}&space;*&space;x_{1}&space;&plus;&space;b_{2}&space;*&space;x_{2}&space;&plus;&space;...&space;&plus;&space;s" title="y = b_{0} + b_{1} * x_{1} + b_{2} * x_{2} + ... + s" /></a>

trong đó: 
* $y$: biến phụ thuộc (biến ta sẽ dự đoán giá trị)
* $x_{1}, x_{2}, ...$: biến độc lập (tác động lên y)
* $b_{1}, b_{2}, ...$: độ dốc (mức độ thay đổi của y khi x thay đổi 1 đơn vị)
* $s$: sai số
* $b_{0}$: giá trị chặn (intercept)

![](https://www.mometrix.com/blog/wp-content/uploads/2020/10/unnamed.png)



Một nhược điểm của mô hình này là nó rất nhạy cảm với dữ liệu bất thường (outliners). Trong trường hợp này đã được khắc phục khi đưa về dự đoán log (đồ thị khảo sát giá đã làm).

*Update: do thiết bị không đủ RAM và kaggle giới hạn thời gian training để chạy toàn bộ tập dữ liệu nên mô hình này chưa thể cho ra kết quả*.



In [ ]:
# model = LinearRegression()
# file_name = 'linear_regression.sav'

## TRAINING
# train(model, file_name)

# model = load(file_name)

## EVALUATE
# result = loss(model, X_test, y_test)

# if(result < rmsle_result):
#     rmsle_result = result
#     best_model = file_name

## Ridge Regression

Về cơ bản cũng giống với Linear Regression truyền thống, nhưng có điểm khác là công thức tìm giá trị hệ số b thì bổ sung thêm ràng buộc để sao cho các hệ số b nhỏ nhất có thể đến mức gần bằng 0, nghĩa là các feature (x) ít có ảnh hưởng tới giá trị đầu ra.

Lambda còn gọi là tham số Regularization, hay tham số Penalty, là số luôn dương (nghĩa là hệ số b càng to loss càng lớn)

![](https://www.analyticsinsight.net/wp-content/uploads/2017/09/ridge_regression_geomteric.png)

Trên hình là giá trị hệ số b1, b2 theo OLS (linear regression truyền thống) và theo Ridge.

Các hệ số theo Ridge sẽ nhỏ hơn đáng kể so với OLS, dẫn đến đa phần training score sẽ thấp hơn nhưng điểm test thực tế lại cao hơn (so với linear regression truyền thống). Vì Ridge đã loại bớt được số lượng các hệ số, giảm bớt độ phức tạp của mô hình, qua đó tránh được tình trạng overfitting, đạt được genelization với dữ liệu test.

In [ ]:
model = Ridge()
file_name = 'ridge.sav'

## TRAINING
# train(model, file_name)

model = load(file_name)

## EVALUATE
result = loss(model, X_test, y_test)

if(result < rmsle_result):
    rmsle_result = result
    best_model = file_name

## SGD Regressor

Với default *loss='squared_loss'*, model đang sử dụng là Linear Regression truyền thống với thuật toán tối ưu hàm mất mát Stochastic Gradient Descent.

Việc tìm global minimum của hàm mất mát gần như bất khả thi. Thay vào đó ta tìm điểm local minimum có giá trị chấp nhận được rồi coi đó là kết quả của bài toán. Hướng tiếp cận ta đề cập đến là Gradient Descent: xuất phát từ một điểm mà chúng ta coi là *gần* với nghiệm của bài toán, sau đó dùng một phép toán lặp để *tiến dần* đến điểm cần tìm, tức đến khi đạo hàm gần với 0.

![](https://1.bp.blogspot.com/-GQOE2Jf92oA/XNQDNLoQl7I/AAAAAAAAA1I/ifpzMryE2zgx7Y7SGy2IS4nTSNbE2EgggCLcBGAs/s1600/theta_new.png)

Hàm mất mát của Linear Regression là:

![](https://upload.wikimedia.org/wikipedia/commons/8/89/Linear_Regression_Loss_Function.png)

Gradient Descent

Tìm local minimum bằng cách cập nhật $w_{t}$ (vector các hệ số), dấu trừ thể hiện việc phải đi ngược với đạo hàm: 

<a href="https://www.codecogs.com/eqnedit.php?latex=w_{t&plus;1}&space;=&space;w_{t}&space;-&space;\Delta&space;*&space;{loss}'(w)" target="_blank"><img src="https://latex.codecogs.com/gif.latex?w_{t&plus;1}&space;=&space;w_{t}&space;-&space;\Delta&space;*&space;{loss}'(w)" title="w_{t+1} = w_{t} - \Delta * {loss}'(w)" /></a>

trong đó:
* $\Delta$: tốc độ học (learning rate)
* ${loss}'(w)$: đạo hàm của hàm loss theo w

Stochastic Gradient Descent

Thay vì tính đạo hàm hàm loss trên toàn bộ tập dữ liệu rồi mới cập nhật w, giờ đây ta cập nhật trên mỗi điểm dữ liệu. Việc cập nhật từng điểm một như thế này có thể làm giảm đi tốc độ thực hiện 1 epoch (một lần duyệt qua toàn bộ các điểm trên tập dữ liệu). Nhưng mặt khác, SGD chỉ yêu cầu một lượng epoch rất nhỏ. Vì vậy SGD phù hợp với các bài toán có lượng data lớn.

In [ ]:
model = SGDRegressor()
file_name = 'sgd_regressor.sav'

## TRAINING
# train(model, file_name)

model = load(file_name)

## EVALUATE
result = loss(model, X_test, y_test)

if(result < rmsle_result):
    rmsle_result = result
    best_model = file_name

## Ensemble models (Boosting method)

**Cơ chế hoạt động**:

Mỗi base model được gọi là một weak learner. Chúng sẽ không hoạt động tốt trên toàn bộ tập D, nhưng khi kết hợp nhiều weak learners ta được một strong learner. Strong learner này chắc chắn sẽ hiệu quả trên tập D.

(Một model thuộc nhóm **Bagging** là RandomForestRegressor đã được thử nhưng vượt quá 9 tiếng kaggle cho phép)

### XGBoost
Sử dụng nhiều base model là Decision Tree, làm mịn Training Loss (Sai số khi huấn luyện) và Regularization (Chuẩn hóa sai số, hệ số và số biến)



Mang nhiều ưu điểm vượt trội như:
* Tốc độ xử lý
* Cơ chế regularization xử lý overfit
* Tự động bỏ qua nodes không mang giá trị tích cực trong việc mở rộng Tree
* ...

Chính vì những ưu điểm đó mà hiệu năng của XGBoost tăng lên đáng kể so với các thuật toán ensemble learning khác.

**Kết quả trả về của model không được tốt!** 

Root Mean Squared Error: 0.5341069414140582

<a href="#Conclusion">Giải thích về kết quả của XGBoost</a>


Trên thực tế, có một model được đánh giá tốt hơn XGBoost trong việc xử lý lượng data lớn là Light GBM.

### Light GBM

Light GBM đánh bại tất cả các thuật toán khác khi tập dataset có kích thước cực lớn. Thực tế chứng minh, nó cần ít thời gian đê xử lý hơn trên tập dữ liệu này. Nguyên nhân sâu xa của sự khác biệt này nằm ở cơ chế làm viêc của Light GBM. Trong khi các thuật toán khác sử dụng cơ chế level-wise thì nó lại sử dụng leaf-wise.

![](https://rohitgr7.github.io/content/images/2019/03/Screenshot-from-2019-03-27-23-09-47-1.png)

Như chúng ta thấy, leaf-wise chỉ mở rộng tree theo 1 trong 2 hướng so với cả 2 hướng của level-wise, tức là số lượng tính toán của Light GBM chỉ bằng 1/2 so với XGBoost.

In [ ]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
file_name = 'lgbm.sav'

## TRAINING
# train(model, file_name)

model = load(file_name)

## EVALUATE
result = loss(model, X_test, y_test)

if(result < rmsle_result):
    rmsle_result = result
    best_model = file_name

Performance của Light GBM còn kém hơn cả XGBoost. Có lẽ Light GBM chỉ mạnh hơn XGBoost ở tốc độ tính toán mà thôi.

# Conclusion

**Tham khảo từ:**

[**Kaggle discussion**: điểm mạnh và điểm yếu của XGBoost.](https://www.kaggle.com/discussion/196542)

[**Medium post**: những bài toán không phải điểm mạnh của XGBoost và cách giải quyết.](https://towardsdatascience.com/why-xgboost-cant-solve-all-your-problems-b5003a62d12a)

**Vấn đề:**

XGBoost được đánh giá vô cùng mạnh mẽ khi có thể giải quyết các bài toán phi tuyến tính cùng các kỹ thuật như regularization để giảm overfiting dữ liệu.

Nhưng bên cạnh đó, XGBoost nói riêng và các thuật toán tree-based nói chung vẫn không tránh khỏi một vài hạn chế, đặc biệt là trong việc giải quyết bài toán xử lý ngôn ngữ.

Trích nguyên văn từ trang web tham khảo:

***When to NOT use XGBoost?***
* *Natural language processing*
* *Regression tasks that involve predicting a continuous output.*

Bài toán của ta bao gồm cả 2 nhiệm vụ trên: xử lý ngôn ngữ và dự đoán giá trị liên tục - đều không phải điểm mạnh của XGBoost.

**Giải pháp:**

Cách giải quyết được đề ra là sử dụng Neural Network với khả năng fit đa dạng loại dữ liệu và cho phép *capture complex trends in data.*

*Đã huấn luyện trên model MLPRegressor nhưng không thành công do vượt quá memory.*

**Future work:**

Sẽ huấn luyện tập dự liệu này bằng Neural Network khi điều kiện về thiết bị và thời gian cho phép!

# Result
Dựa vào đánh giá trên tập valid, chọn model tốt nhất để đưa ra kết quả cuối cùng:

In [ ]:
submission = pd.read_csv('../working/sample_submission_stg2.csv')

model = pickle.load(open(os.path.join('/kaggle/input/trained-model', best_model), 'rb'))
print('Use ' + best_model + ' model')
preds = model.predict(test_features)
preds = np.exp(preds) - 1
submission.loc[:, 'price'] = preds
submission

In [ ]:
submission.to_csv('submission.csv', index=False)